<a href="https://colab.research.google.com/github/sunshineluyao/simulations/blob/main/Simulation_ExecuteAsExternalPythonFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# References

[Python Generator Functions using yield](https://www.datacamp.com/community/news/python-generators-a-complete-guide-to-create-and-use-generators-6v0phby17gw)

[Courses on Real Python](https://realpython.com/courses/simulating-processes-simpy/)

[Google Colab Tutorial | How to Execute external Python (.py) File](https://youtu.be/fPDGu1kQCzE)

[Mount Drive](https://www.tutorialspoint.com/google_colab/google_colab_executing_external_python_files.htm)


#Method 1: Execute the .py file in google folder


*   Please note in google folder setting, uncheck "automatically change the uploaded files to google doc"
*   Please note don't download the file as a name with simpy"


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!ls "/content/drive/My Drive/Python"

simulation.py


In [22]:
!pip install simpy

In [24]:
!python3 "/content/drive/My Drive/Python/simulation.py"

0.51.2
Input # of cashiers working:100
Input # of servers working:100
Input # of ushers working:100
Running simulation... 
The average wait time is 5 minutes and 33 seconds.


# Method 2: Compile the file in Colab directly

In [25]:
%%writefile simulation2.py
import simpy
import random
import statistics
import numpy as np
import numba
wait_times =[]
class Theater(object):
  def __init__(self,env,num_cashiers, num_servers, num_ushers):
    self.env = env
    self.cashier = simpy.Resource(env, num_cashiers)
    self.server = simpy.Resource(env, num_servers)
    self.usher = simpy.Resource(env, num_ushers)
  def purchase_ticket(self, moviegoer):
    yield self.env.timeout(random.randint(1,3))
  def check_ticket(self, moviegoer):
    yield self.env.timeout(3/60)
  def sell_food(self, moviegoer):
    yield self.env.timeout(random.randint(1,6))
def go_to_movies(env, moviegoer, theater):
  # arrive at theater
  arrival_time = env.now
  #buy ticket
  with theater.cashier.request() as request:
    yield request
    yield env.process(theater.purchase_ticket(moviegoer))
  #check ticket
  with theater.usher.request() as request:
    yield request
    yield env.process(theater.check_ticket(moviegoer))
  # buy food
  if random.choice([True, False]):
    with theater.server.request() as request:
      yield request
      yield env.process(theater.sell_food(moviegoer))
    # go to their seat
    wait_times.append(env.now-arrival_time)
    return wait_times
def run_theater(env, num_cashiers, num_servers, num_ushers):
  theater = Theater(env, num_cashiers, num_servers, num_ushers)
  for moviegoer in range(3):
    env.process(go_to_movies(env, moviegoer, theater))

  while True:
    yield env.timeout(0.20)

    moviegoer +=1
    env.process(go_to_movies(env, moviegoer, theater))
def calculate_wait_times(wait_times):
  average_wait =statistics.mean(wait_times)
  #pretty print results:
  minutes, frac_minutes = divmod(average_wait,1)
  seconds = frac_minutes*60
  return round(minutes), round(seconds)
def get_user_input():
  num_cashiers = input("Input # of cashiers working:")
  num_servers = input("Input # of servers working:")
  num_ushers = input("Input # of ushers working:")

  params = [num_cashiers, num_servers, num_ushers]

  if all(str(i).isdigit() for i in params):
    params = [int(x) for x in params]
  else:
    print(
        "Could not parse input. The simulation will use default values"
        "\n1 casher, 1 server, 1 usher."
    )
    params = [1,1,1]
  return params
def main():
  #Setup
  random.seed(42)
  num_cashiers, num_servers, num_ushers = get_user_input()
  #run the simulation
  env =simpy.Environment()
  env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
  env.run(until=90) ###minutes within the environment

  #view the results
  mins, secs =calculate_wait_times(wait_times)
  print(
      "Running simulation...",
      f"\nThe average wait time is {mins} minutes and {secs} seconds.",
  )
if __name__ =='__main__':
  main()

Writing simulation2.py


### copy path from the left files window and execute

In [26]:
!python /content/simulation2.py

Input # of cashiers working:50
Input # of servers working:30
Input # of ushers working:10
Running simulation... 
The average wait time is 5 minutes and 33 seconds.
